In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Inches
import matplotlib.pyplot as plt
import pandas as pd

ruta_archivo = '/content/base_1.xlsx'

# Leer hoja EN_TRAMITE
df_en_tramite = pd.read_excel(ruta_archivo, sheet_name='EN_TRAMITE')

# Crear documento Word
doc = Document()
doc.add_heading('Reporte de registros CITES', 0)

# Función para formatear números con separador de miles y 2 decimales si aplica
def format_number(x):
    if isinstance(x, float) and not x.is_integer():
        return f"{x:,.2f}"
    else:
        return f"{int(x):,}"

# Función para poner bordes a tabla Word
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls

def set_table_border(table):
    tbl = table._tbl
    tblPr = tbl.tblPr
    borders = parse_xml(r'''
    <w:tblBorders %s>
      <w:top w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:left w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:bottom w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:right w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:insideH w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:insideV w:val="single" w:sz="8" w:space="0" w:color="000000"/>
    </w:tblBorders>''' % nsdecls('w'))
    tblPr.append(borders)

# Función para envolver texto largo en etiquetas (dos líneas)
def wrap_label(label, max_width=15):
    if len(label) <= max_width:
        return label
    else:
        mid = len(label) // 2
        before = label.rfind(' ', 0, mid)
        after = label.find(' ', mid)
        split_pos = before if before != -1 else after
        if split_pos == -1:
            split_pos = mid
        return label[:split_pos] + '\n' + label[split_pos+1:]

# --- Texto introductorio ---
total_registros = len(df_en_tramite)

p = doc.add_paragraph(
    f"Esta sección presenta los datos consolidados de registros en trámite según diferentes variables: empresa, especie, estado y profesional responsable. De un total de {total_registros:,} registros pendiente de atención se tiene la siguiente información:"
)
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
# --- Gráfico 1: registros por empresa ---
registros_por_empresa = df_en_tramite['Administrado'].value_counts()

doc.add_heading('Cantidad de registros por empresa', level=1)
h=doc.add_paragraph(
    "El siguiente gráfico muestra la cantidad de registros clasificados por empresa:"
)
h.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Preparar etiquetas envueltas para las empresas
labels_wrapped = [wrap_label(label) for label in registros_por_empresa.index]
plt.figure(figsize=(14,8))
ax = registros_por_empresa.plot(kind='bar')
plt.title('Gráfico N°01:Cantidad de registros por empresa', fontsize=20)
plt.xlabel('Empresa', fontsize=16)
plt.ylabel('Cantidad de registros', fontsize=16)
plt.xticks(ticks=range(len(labels_wrapped)), labels=labels_wrapped, rotation=90, fontsize=14)
plt.yticks(fontsize=14)

for p in ax.patches:
    valor = p.get_height()
    ax.annotate(format_number(valor),
                (p.get_x() + p.get_width() / 2, valor),
                ha='center', va='bottom', fontsize=16)
plt.tight_layout()
plt.savefig('/content/grafico1.png')
plt.close()
doc.add_picture('/content/grafico1.png', width=Inches(7))

# --- Gráfico 2: especie por kg excluyendo raya amazonica y sin valores nulos o cero ---
df_en_tramite['Cantidad_aletas_ secas_kg'] = pd.to_numeric(df_en_tramite['Cantidad_aletas_ secas_kg'], errors='coerce')
df_filtrado = df_en_tramite[~df_en_tramite['Especie'].str.lower().eq('raya amazonica')].copy()
df_filtrado = df_filtrado[(df_filtrado['Cantidad_aletas_ secas_kg'].notna()) & (df_filtrado['Cantidad_aletas_ secas_kg'] > 0)]
especies_kg = df_filtrado.groupby('Especie')['Cantidad_aletas_ secas_kg'].sum().sort_values(ascending=False)
doc.add_heading("Cantidad de kilogramos por especie", level=1)
r=doc.add_paragraph(
    "El siguiente gráfico muestra la cantidad total en kilogramos para cada especie. Considerar que para el caso de 'raya amazónicas' son unidades no kilogramos."
)
r.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

plt.figure(figsize=(14,8))
ax = especies_kg.plot(kind='bar')
plt.title('Gráfico N°02:Cantidad de kilogramos por especie', fontsize=20)
plt.xlabel('Especie', fontsize=16)
plt.ylabel('Kilogramos', fontsize=16)
plt.xticks(fontsize=14, rotation=90)  # Etiquetas verticales
plt.yticks(fontsize=14)

for p in ax.patches:
    valor = p.get_height()
    ax.annotate(format_number(valor),
                (p.get_x() + p.get_width() / 2, valor),
                ha='center', va='bottom', fontsize=16)
plt.tight_layout()
plt.savefig('/content/grafico2.png')
plt.close()

doc.add_picture('/content/grafico2.png', width=Inches(7))

# --- Gráfico 3: Estado clasificado ---
def clasificar_estado(estado):
    if isinstance(estado, str):
        if estado.startswith('Pdte'):
            return 'Pendiente'
        elif estado in ['Observado', 'Reobservado']:
            return estado
        elif estado.startswith('Elevado'):
            return 'Elevado DVPA'
        else:
            return 'Evaluacion'
    else:
        return 'Evaluacion'

df_en_tramite['Estado_Clasificado'] = df_en_tramite['Estado'].apply(clasificar_estado)
estados_count = df_en_tramite['Estado_Clasificado'].value_counts()

doc.add_heading("Cantidad según el estado del trámite", level=1)
t=doc.add_paragraph(
    "El siguiente gráfico muestra la cantidad de registros agrupados según la situación actual del trámite del regsitro: en Pendiente respuesta externa, Observado, Reobservado, Evaluación y Elvado a DVPA."
)
t.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
plt.figure(figsize=(14,8))
ax = estados_count.plot(kind='barh')
plt.title('Gráfico N°03:Cantidad según estado del trámite', fontsize=20)
plt.xlabel('Cantidad', fontsize=16)
plt.ylabel('Estado', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

for p in ax.patches:
    valor = p.get_width()
    ax.annotate(format_number(valor),
                (valor, p.get_y() + p.get_height()/2),
                ha='left', va='center', fontsize=16)
plt.tight_layout()
plt.savefig('/content/grafico3.png')
plt.close()

doc.add_picture('/content/grafico3.png', width=Inches(7))

# --- Tabla resumen por profesional ---
tabla_profesional = df_en_tramite.groupby('Profesional').agg(
    cantidad_registros = ('Profesional','count'),
    cantidad_cdts = ('Cantidad_CDTS','sum')
).reset_index()

doc.add_heading('Tabla resumen por Profesional', level=1)
e=doc.add_paragraph(
    "La siguiente tabla muestra la distribución total de registros pendientes y la cantidad de CDTs asignados a cada profesional."
)
e.alignment=WD_ALIGN_PARAGRAPH.JUSTIFY

tabla = doc.add_table(rows=1, cols=3)
hdr_cells = tabla.rows[0].cells
hdr_cells[0].text = 'Profesional'
hdr_cells[1].text = 'Cantidad de registros'
hdr_cells[2].text = 'Cantidad CDTS'

for index, row in tabla_profesional.iterrows():
    row_cells = tabla.add_row().cells
    row_cells[0].text = str(row['Profesional'])
    row_cells[1].text = str(row['cantidad_registros'])
    row_cells[2].text = str(row['cantidad_cdts'])

# Aplicar borde a la tabla
set_table_border(tabla)

# Guardar documento
ruta_guardado = '/content/reporte_cites.docx'
doc.save(ruta_guardado)

print(f'Reporte generado y guardado en: {ruta_guardado}')


Reporte generado y guardado en: /content/reporte_cites.docx


In [ ]:
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Inches
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

ruta_archivo = '/content/base_1.xlsx'

# Función para formatear números con separador de miles y 2 decimales si aplica
def format_number(x):
    if isinstance(x, float) and not x.is_integer():
        return f"{x:,.2f}"
    else:
        return f"{int(x):,}"

# Función para poner bordes a tabla Word
def set_table_border(table):
    tbl = table._tbl
    tblPr = tbl.tblPr
    borders = parse_xml(r'''
    <w:tblBorders %s>
      <w:top w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:left w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:bottom w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:right w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:insideH w:val="single" w:sz="8" w:space="0" w:color="000000"/>
      <w:insideV w:val="single" w:sz="8" w:space="0" w:color="000000"/>
    </w:tblBorders>''' % nsdecls('w'))
    tblPr.append(borders)

# Función para envolver texto largo en etiquetas (dos líneas)
def wrap_label(label, max_width=15):
    if len(label) <= max_width:
        return label
    else:
        mid = len(label) // 2
        before = label.rfind(' ', 0, mid)
        after = label.find(' ', mid)
        split_pos = before if before != -1 else after
        if split_pos == -1:
            split_pos = mid
        return label[:split_pos] + '\n' + label[split_pos+1:]

# --- LEER DATOS ---
df_en_tramite = pd.read_excel(ruta_archivo, sheet_name='EN_TRAMITE')
df_resoluciones = pd.read_excel(ruta_archivo, sheet_name='RESOLUCIONES')

# Crear documento Word
doc = Document()
doc.add_heading('Reporte de registros CITES', 0)

##########################
# SECCIÓN 1: EN_TRAMITE
##########################

total_registros = len(df_en_tramite)

p = doc.add_paragraph(
    f"Esta sección presenta los datos consolidados de registros en trámite según diferentes variables: empresa, especie, estado y profesional responsable. De un total de {total_registros:,} registros pendiente de atención se tiene la siguiente información:"
)
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Gráfico 1: registros por empresa
registros_por_empresa = df_en_tramite['Administrado'].value_counts()

doc.add_heading('Cantidad de registros por empresa', level=1)
h = doc.add_paragraph(
    "El siguiente gráfico muestra la cantidad de registros clasificados por empresa:"
)
h.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

labels_wrapped = [wrap_label(label) for label in registros_por_empresa.index]
plt.figure(figsize=(14,8))
ax = registros_por_empresa.plot(kind='bar')
plt.title('Gráfico N°01:Cantidad de registros por empresa', fontsize=20)
plt.xlabel('Empresa', fontsize=16)
plt.ylabel('Cantidad de registros', fontsize=16)
plt.xticks(ticks=range(len(labels_wrapped)), labels=labels_wrapped, rotation=90, fontsize=14)
plt.yticks(fontsize=14)
for p in ax.patches:
    valor = p.get_height()
    ax.annotate(format_number(valor), (p.get_x() + p.get_width() / 2, valor),
                ha='center', va='bottom', fontsize=16)
plt.tight_layout()
plt.savefig('/content/grafico1.png')
plt.close()
doc.add_picture('/content/grafico1.png', width=Inches(7))

# Gráfico 2: especie por kg excluyendo raya amazonica y sin valores nulos o cero
df_en_tramite['Cantidad_aletas_ secas_kg'] = pd.to_numeric(df_en_tramite['Cantidad_aletas_ secas_kg'], errors='coerce')
df_filtrado = df_en_tramite[~df_en_tramite['Especie'].str.lower().eq('raya amazonica')].copy()
df_filtrado = df_filtrado[(df_filtrado['Cantidad_aletas_ secas_kg'].notna()) & (df_filtrado['Cantidad_aletas_ secas_kg'] > 0)]
especies_kg = df_filtrado.groupby('Especie')['Cantidad_aletas_ secas_kg'].sum().sort_values(ascending=False)

doc.add_heading("Cantidad de kilogramos por especie", level=1)
r = doc.add_paragraph(
    "El siguiente gráfico muestra la cantidad total en kilogramos para cada especie. Considerar que para el caso de 'raya amazónicas' son unidades no kilogramos."
)
r.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

plt.figure(figsize=(14,8))
ax = especies_kg.plot(kind='bar')
plt.title('Gráfico N°02:Cantidad de kilogramos por especie', fontsize=20)
plt.xlabel('Especie', fontsize=16)
plt.ylabel('Kilogramos', fontsize=16)
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
for p in ax.patches:
    valor = p.get_height()
    ax.annotate(format_number(valor), (p.get_x() + p.get_width() / 2, valor),
                ha='center', va='bottom', fontsize=16)
plt.tight_layout()
plt.savefig('/content/grafico2.png')
plt.close()
doc.add_picture('/content/grafico2.png', width=Inches(7))

# Gráfico 3: Estado clasificado
def clasificar_estado(estado):
    if isinstance(estado, str):
        if estado.startswith('Pdte'):
            return 'Pendiente'
        elif estado in ['Observado', 'Reobservado']:
            return estado
        elif estado.startswith('Elevado'):
            return 'Elevado DVPA'
        else:
            return 'Evaluacion'
    else:
        return 'Evaluacion'

df_en_tramite['Estado_Clasificado'] = df_en_tramite['Estado'].apply(clasificar_estado)
estados_count = df_en_tramite['Estado_Clasificado'].value_counts()

doc.add_heading("Cantidad según el estado del trámite", level=1)
t = doc.add_paragraph(
    "El siguiente gráfico muestra la cantidad de registros agrupados según la situación actual del trámite del registro: en Pendiente respuesta externa, Observado, Reobservado, Evaluación y Elevado a DVPA."
)
t.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

plt.figure(figsize=(14,8))
ax = estados_count.plot(kind='barh')
plt.title('Gráfico N°03:Cantidad según estado del trámite', fontsize=20)
plt.xlabel('Cantidad', fontsize=16)
plt.ylabel('Estado', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
for p in ax.patches:
    valor = p.get_width()
    ax.annotate(format_number(valor), (valor, p.get_y() + p.get_height() / 2),
                ha='left', va='center', fontsize=16)
plt.tight_layout()
plt.savefig('/content/grafico3.png')
plt.close()
doc.add_picture('/content/grafico3.png', width=Inches(7))

# Tabla resumen por profesional
tabla_profesional = df_en_tramite.groupby('Profesional').agg(
    cantidad_registros=('Profesional', 'count'),
    cantidad_cdts=('Cantidad_CDTS', 'sum')
).reset_index()

doc.add_heading('Tabla resumen por Profesional', level=1)
e = doc.add_paragraph(
    "La siguiente tabla muestra la distribución total de registros pendientes y la cantidad de CDTs asignados a cada profesional."
)
e.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

tabla = doc.add_table(rows=1, cols=3)
hdr_cells = tabla.rows[0].cells
hdr_cells[0].text = 'Profesional'
hdr_cells[1].text = 'Cantidad de registros'
hdr_cells[2].text = 'Cantidad CDTS'
for index, row in tabla_profesional.iterrows():
    row_cells = tabla.add_row().cells
    row_cells[0].text = str(row['Profesional'])
    row_cells[1].text = str(row['cantidad_registros'])
    row_cells[2].text = str(row['cantidad_cdts'])
set_table_border(tabla)

##########################
# SECCIÓN 2: RESOLUCIONES
##########################

# Filtrar registros del año 2025
df_resoluciones_2025 = df_resoluciones[df_resoluciones['año_RD'] == 2025]

total_rds_2025 = len(df_resoluciones_2025)
doc.add_page_break()
doc.add_heading("Sección de registros CITES atendidos", level=0)
p2 = doc.add_paragraph(
    f"De un total de {total_rds_2025:,} resoluciones del año 2025, estas se distribuyen en los siguientes meses:"
)
p2.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Gráfico cantidad de RDs por mes_RD
rds_por_mes = df_resoluciones_2025['mes_RD'].value_counts().sort_index()
plt.figure(figsize=(14,8))
ax = rds_por_mes.plot(kind='bar')
plt.title('Cantidad de Resoluciones por Mes (2025)', fontsize=20)
plt.xlabel('Mes', fontsize=16)
plt.ylabel('Cantidad de Resoluciones', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
for p in ax.patches:
    valor = p.get_height()
    ax.annotate(f"{int(valor):,}", (p.get_x() + p.get_width() / 2, valor),
                ha='center', va='bottom', fontsize=16)
plt.tight_layout()
plt.savefig('/content/grafico_rds_mes.png')
plt.close()
doc.add_picture('/content/grafico_rds_mes.png', width=Inches(7))

# Gráfico procedentes/improcedentes por Pedido
df_rds_filtrado = df_resoluciones_2025[
    (df_resoluciones_2025['Resultado'].isin(['PROCEDENTE', 'IMPROCEDENTE'])) &
    (df_resoluciones_2025['Pedido'].isin(['Exportación', 'Re exportación', 'Sustitución', 'Importación']))
]

tabla_resultado_pedido = pd.crosstab(
    index=df_rds_filtrado['Pedido'],
    columns=df_rds_filtrado['Resultado']
)[['PROCEDENTE', 'IMPROCEDENTE']]

plt.figure(figsize=(8,4))
ax = tabla_resultado_pedido.plot(kind='bar', stacked=False)
plt.title('Cantidad de Procedentes e Improcedentes por Pedido (2025)', fontsize=12)
plt.xlabel('Pedido', fontsize=10)
plt.ylabel('Cantidad', fontsize=10)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
for container in ax.containers:
    for patch in container.patches:
        height = patch.get_height()
        if height > 0:
            ax.annotate(f"{int(height):,}", (patch.get_x() + patch.get_width() / 2, height),
                        ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.savefig('/content/grafico_resultado_pedido.png')
plt.close()

doc.add_heading('Distribución de resultados por tipo de pedido', level=1)
p3 = doc.add_paragraph(
    "El siguiente gráfico muestra la cantidad de resoluciones clasificadas como Procedentes e Improcedentes, distribuidas por tipo de Pedido: Exportación, Re exportación, Sustitución e Importación."
)
p3.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
doc.add_picture('/content/grafico_resultado_pedido.png', width=Inches(7))

# Gráfico tiempo respuesta por Pedido
df_tiempo = df_rds_filtrado[df_rds_filtrado['Tiempo_respuesta\n(dias_habiles)'].notna()]
order = ['Exportación', 'Re exportación', 'Sustitución', 'Importación']
data_to_plot = [df_tiempo.loc[df_tiempo['Pedido'] == p, 'Tiempo_respuesta\n(dias_habiles)'] for p in order]

plt.figure(figsize=(14,8))
bp = plt.boxplot(data_to_plot, labels=order, patch_artist=True)
plt.title('Tiempo de respuesta (días hábiles) por Pedido (2025)', fontsize=20)
plt.xlabel('Pedido', fontsize=16)
plt.ylabel('Días hábiles', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

means = [np.mean(d) if len(d) > 0 else 0 for d in data_to_plot]
for i, mean in enumerate(means):
    plt.text(i + 1, mean, f'{mean:.2f}', ha='center', fontsize=16, color='red',fontweight='bold')

plt.tight_layout()
plt.savefig('/content/grafico_tiempo_respuesta.png')
plt.close()

doc.add_heading('Tiempo de respuesta por Pedido', level=1)
p4 = doc.add_paragraph(
    "El gráfico muestra la distribución del tiempo de respuesta en días hábiles para cada tipo de Pedido."
)
p4.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
doc.add_picture('/content/grafico_tiempo_respuesta.png', width=Inches(7))

# Guardar documento final
doc.save('/content/reporte_cites_completo.docx')
print('Reporte completo generado en /content/reporte_cites_completo.docx')


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
<ipython-input-4-2182efb73800>:258: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(data_to_plot, labels=order, patch_artist=True)


Reporte completo generado en /content/reporte_cites_completo.docx


<Figure size 800x400 with 0 Axes>

In [ ]:
print(df_rds_filtrado.columns.tolist())

['Registro', 'Fecha_registro', 'Administrado', 'RD', 'Fecha_RD', 'Tiempo_respuesta\n(dias_habiles)', 'Pedido', 'Permiso CITES', 'Estampilla', 'Resultado', 'Especie', 'CDTS', 'Cantidad autorizada\n(kg)', 'Cantidad refrendada\n(kg)', 'Analista', 'Sumilla', 'mes_RD', 'año_RD', 'Unnamed: 18']
